In [1]:
import numpy as np #calculation of lenear Algebra
import pandas as pd #data frame (data processing, CSV file I/O) (e.g. pd.read_csv)
import seaborn as sns #data visualization
from matplotlib import pyplot as plt #data visualization
%matplotlib inline
import os #interact with the underlying operating system
# supress/hide warnings
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc

In [2]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PDFMalware2022.csv")
df.isnull().sum()
new_df=df.dropna(axis=0,how='any')

Mounted at /content/drive


In [3]:
print("Old Dataframe length: ", len(df))
print("New Dataframe lenght: ", len(new_df))
print("Number of rows with at least 1 Null Values: ",(len(df)-len(new_df)))

Old Dataframe length:  10026
New Dataframe lenght:  10023
Number of rows with at least 1 Null Values:  3


In [4]:
print(new_df['text'].value_counts())
print(len(new_df))

new_df = new_df[new_df.text !='-1']
new_df = new_df[new_df.text !='0']
new_df = new_df[new_df.text !='unclear']


No         5398
Yes        3761
unclear     549
-1          302
0            13
Name: text, dtype: int64
10023


In [5]:
new_df['text'].replace(['No','Yes','unclear', '-1', '0'],[0,1,2,3,4],inplace=True)
print(new_df['text'].value_counts())
print(len(new_df))

print("Clas names converted to 1 and 0:")
new_df['Class'].replace(['Malicious','Benign'],[1,0],inplace=True)
print(new_df['Class'].value_counts())

0    5398
1    3761
Name: text, dtype: int64
9159
Clas names converted to 1 and 0:
1    4696
0    4463
Name: Class, dtype: int64


In [6]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
fit_df = new_df.apply(le.fit_transform)
fit_df.describe()

,Fine name,pdfsize,metadata size,pages,xref Length,title characters,isEncrypted,embedded files,images,text,...,AA,OpenAction,Acroform,JBIG2Decode,RichMedia,launch,EmbeddedFile,XFA,Colors,Class
count,9159.000000,9159.000000,9159.000000,9159.000000,9159.000000,9159.000000,9159.000000,9159.000000,9159.000000,9159.000000,...,9159.000000,9159.000000,9159.000000,9159.000000,9159.000000,9159.000000,9159.000000,9159.000000,9159.000000,9159.000000
mean,4579.000000,68.026968,77.714707,3.399825,121.129053,9.334534,0.007534,0.020635,4.803035,0.410634,...,1.193253,1.258325,1.440878,1.018670,0.957965,0.964843,1.840703,1.004804,1.699749,0.512720
std,2644.119891,91.431103,74.832570,7.164753,217.117658,18.654907,0.086473,0.153257,14.287965,0.491976,...,2.229709,0.584693,0.957022,0.773116,0.244355,0.253105,3.475440,0.344328,4.173334,0.499865
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2289.500000,9.000000,1.000000,1.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
50%,4579.000000,37.000000,62.000000,1.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,6868.500000,83.000000,107.000000,2.000000,86.000000,13.000000,0.000000,0.000000,1.000000,1.000000,...,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,9158.000000,490.000000,388.000000,83.000000,901.000000,147.000000,1.000000,3.000000,86.000000,1.000000,...,39.000000,6.000000,8.000000,15.000000,5.000000,4.000000,17.000000,5.000000,58.000000,1.000000


In [7]:
fit_df.drop(['Fine name','pageno','AA','JBIG2Decode', 'launch','encrypt','embedded files', 'isEncrypted', 'RichMedia', 'Acroform','EmbeddedFile','title characters','Colors'],inplace=True,axis=1)

In [8]:
fit_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9159 entries, 0 to 10025
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   pdfsize        9159 non-null   int64
 1   metadata size  9159 non-null   int64
 2   pages          9159 non-null   int64
 3   xref Length    9159 non-null   int64
 4   images         9159 non-null   int64
 5   text           9159 non-null   int64
 6   header         9159 non-null   int64
 7   obj            9159 non-null   int64
 8   endobj         9159 non-null   int64
 9   stream         9159 non-null   int64
 10  endstream      9159 non-null   int64
 11  xref           9159 non-null   int64
 12  trailer        9159 non-null   int64
 13  startxref      9159 non-null   int64
 14  ObjStm         9159 non-null   int64
 15  JS             9159 non-null   int64
 16  Javascript     9159 non-null   int64
 17  OpenAction     9159 non-null   int64
 18  XFA            9159 non-null   int64
 19  Class

In [9]:
from sklearn.model_selection import train_test_split

def data_split(data, data_target):
  X,X_test,Y,y_test= train_test_split(data, data_target,test_size=0.2, shuffle = True, random_state = 0)
  X_train,X_val,y_train,y_val = train_test_split(X, Y,test_size=0.1, shuffle = True, random_state = 0)
  print("X_train shape: {}".format(X_train.shape), "y_train shape: {}".format(y_train.shape))
  print("X_val shape: {}".format(X_val.shape), "y_val shape: {}".format(y_val.shape))
  print("X_test shape: {}".format(X_test.shape), "y_test shape: {}".format(y_test.shape))
  return X_train,X_val,X_test,y_train,y_val,y_test

target_name = 'Class'
data_target = fit_df['Class']
data = fit_df.drop(['Class'], axis=1)
X_train,X_val,X_test,y_train,y_val,y_test = data_split(data, data_target)

X_train shape: (6594, 19) y_train shape: (6594,)
X_val shape: (733, 19) y_val shape: (733,)
X_test shape: (1832, 19) y_test shape: (1832,)


In [10]:
import tensorflow as tf
import keras
from keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from sklearn.preprocessing import StandardScaler

In [11]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val=scaler.transform(X_val)
X_test=scaler.transform(X_test)

Hybrid Neural Network and Random Forest

In [12]:
import tensorflow as tf
import keras
from keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from sklearn.preprocessing import StandardScaler


In [13]:

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val=scaler.transform(X_val)
X_test=scaler.transform(X_test)

In [14]:

model = Sequential()
model.add(Dense(64, input_dim =19, activation ="relu"))
model.add(Dropout(0.2))
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(32, activation = "relu"))

model.add(Dense(16, activation = "relu"))
model.add(Dense(8, activation = "relu"))
model.add(Dense(1, activation ="sigmoid"))


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1280      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 32)                2

In [16]:
model.compile(loss= "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"] )

In [17]:
history = model.fit(X_train, y_train,  epochs=100, batch_size=32, shuffle=True, validation_data =(X_val, y_val))

Epoch 1/100
207/207 [==============================] - 5s 4ms/step - loss: 0.2548 - accuracy: 0.9084 - val_loss: 0.1318 - val_accuracy: 0.9659
Epoch 2/100
207/207 [==============================] - 1s 3ms/step - loss: 0.1160 - accuracy: 0.9682 - val_loss: 0.1022 - val_accuracy: 0.9686
Epoch 3/100
207/207 [==============================] - 1s 5ms/step - loss: 0.1019 - accuracy: 0.9713 - val_loss: 0.0735 - val_accuracy: 0.9768
Epoch 4/100
207/207 [==============================] - 1s 4ms/step - loss: 0.0782 - accuracy: 0.9762 - val_loss: 0.0826 - val_accuracy: 0.9809
Epoch 5/100
207/207 [==============================] - 1s 4ms/step - loss: 0.0738 - accuracy: 0.9786 - val_loss: 0.0633 - val_accuracy: 0.9823
Epoch 6/100
207/207 [==============================] - 1s 4ms/step - loss: 0.0643 - accuracy: 0.9816 - val_loss: 0.0737 - val_accuracy: 0.9823
Epoch 7/100
207/207 [==============================] - 1s 4ms/step - loss: 0.0634 - accuracy: 0.9815 - val_loss: 0.0590 - val_accuracy: 0.9850

In [18]:
eval_model=model.evaluate(X_train, y_train)
eval_model



207/207 [==============================] - 0s 1ms/step - loss: 0.0043 - accuracy: 0.9982


[0.00429935147985816, 0.9981801509857178]

In [19]:

#Validation or test accuracy average
eval_model_val=model.evaluate(X_val, y_val)
eval_model_val


23/23 [==============================] - 0s 2ms/step - loss: 0.0374 - accuracy: 0.9918


[0.03742890805006027, 0.9918144345283508]

In [20]:

#Validation or test accuracy average
eval_model_test=model.evaluate(X_test, y_test)
eval_model_test


58/58 [==============================] - 0s 2ms/step - loss: 0.0730 - accuracy: 0.9896


[0.07298652082681656, 0.989628791809082]

In [21]:
y_pred= model.predict(X_test)
y_pred =(y_pred>0.5)
y_pred.shape



58/58 [==============================] - 0s 1ms/step


(1832, 1)

In [22]:
result = confusion_matrix( y_test , y_pred)
result



array([[853,  10],
       [  9, 960]])

In [23]:
from sklearn.metrics import classification_report


print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       863
           1       0.99      0.99      0.99       969

    accuracy                           0.99      1832
   macro avg       0.99      0.99      0.99      1832
weighted avg       0.99      0.99      0.99      1832



In [24]:
feature_extraction_model = Sequential()
for layer in model.layers[:-1]:  # Exclude the last layer
    feature_extraction_model.add(layer)

# Now feature_extraction_model is the modified model without the last classification layer
feature_extraction_model.summary()
# Use feature_extraction_model to extract features from the data

extracted_features_nn_train = feature_extraction_model.predict(X_train)
extracted_features_nn_test = feature_extraction_model.predict(X_test)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1280      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 32)               

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

#extracted_features_nn_train = feature_extraction_model.predict(X_train)
#extracted_features_nn_test = feature_extraction_model.predict(X_test)

# Random Forest Classifier for Classification
#nn_rf_hybrid = RandomForestClassifier(n_estimators=300, max_depth=30)
nn_rf_hybrid = GridSearchCV(estimator=RandomForestClassifier(), param_grid={'max_depth':[10,30], 'n_estimators':[50,100,200,300]}, cv=3).fit(extracted_features_nn_train, y_train)

#nn_rf_hybrid.fit(extracted_features_nn_train, y_train)

acc_nn_rf_hybrid = round(nn_rf_hybrid.score(extracted_features_nn_train, y_train) * 100, 2)
print(acc_nn_rf_hybrid)

# Make predictions on the test set
y_pred = nn_rf_hybrid.predict(extracted_features_nn_test)

# Evaluate the classifier's performance
accuracy = np.mean(y_pred == y_test)
print("Random Forest Classifier Accuracy:", accuracy)

acc_test_rf_nn_hybrid = round(nn_rf_hybrid.score(extracted_features_nn_test, y_test) * 100, 2)
print("Test Accuracy =", acc_test_rf_nn_hybrid, nn_rf_hybrid.best_params_ )

99.98
Random Forest Classifier Accuracy: 0.9896288209606987
Test Accuracy = 98.96 {'max_depth': 10, 'n_estimators': 50}


In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV


# Random Forest Classifier for Classification
nn_rf_hybrid = RandomForestClassifier(n_estimators=100, max_depth=30)
nn_rf_hybrid.fit(extracted_features_nn_train, y_train)

acc_nn_rf_hybrid = round(nn_rf_hybrid.score(extracted_features_nn_train, y_train) * 100, 2)
print(acc_nn_rf_hybrid)

# Make predictions on the test set
y_pred = nn_rf_hybrid.predict(extracted_features_nn_test)

# Evaluate the classifier's performance
accuracy = np.mean(y_pred == y_test)
print("Random Forest Classifier Accuracy:", accuracy)

acc_test_rf_nn_hybrid = round(nn_rf_hybrid.score(extracted_features_nn_test, y_test) * 100, 2)
print("Test Accuracy =", acc_test_rf_nn_hybrid)

100.0
Random Forest Classifier Accuracy: 0.9896288209606987
Test Accuracy = 98.96


In [28]:
from sklearn.tree import DecisionTreeClassifier
base_estimator = DecisionTreeClassifier(max_depth=3)

# Initialize the AdaBoost classifier
from sklearn.ensemble import AdaBoostClassifier
adaboost_classifier = AdaBoostClassifier(base_estimator=base_estimator, random_state=42)

# Define the hyperparameter grid to search over
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.1, 0.01, 0.001],
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(adaboost_classifier, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters from the grid search results
best_params = grid_search.best_params_
print(best_params)

# Initialize the AdaBoost classifier with the best hyperparameters
best_adaboost_classifier = AdaBoostClassifier(
    base_estimator=base_estimator,
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    random_state=42
)

# Train the classifier on the training data
best_adaboost_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_adaboost_classifier.predict(X_test)


{'learning_rate': 0.1, 'n_estimators': 150}


In [29]:
acc_train_adaboost_par = round(best_adaboost_classifier.score(X_train, y_train) * 100, 2)
print(acc_train_adaboost_par)
acc_val_adaboost_par = round(best_adaboost_classifier.score(X_val, y_val) * 100, 2)
print(acc_val_adaboost_par)
acc_test_adaboost_par = round(best_adaboost_classifier.score(X_test, y_test) * 100, 2)
print(acc_test_adaboost_par)

100.0
99.73
99.51
